In [ ]:
import numpy as np
import pandas as pd
import datetime
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
train.head()

In [ ]:
def split_month(date):
    return date.split('-')[1]

def split_day(date):
    return date.split('-')[2]

def weekend(date):
    weekend = []
    a = pd.to_datetime(date)
    for i in range(len(a)):
        if a.iloc[i].weekday() >= 5 :
            weekend.append(1)
        else:
            weekend.append(0)
    return weekend

def weekday(date):
    weekday = []
    a = pd.to_datetime(date)
    for i in range(len(a)):
        weekday.append(a.iloc[i].weekday())
    return weekday

In [ ]:
train['Month'] = train['date'].apply(split_month)
train['Day'] = train['date'].apply(split_day)
train['Weekend'] = weekend(train['date'])
train['Weekday'] = weekday(train['date'])

train.head()

In [ ]:
train.drop(['row_id' , 'date'] , axis = 1 , inplace  = True)
train.head()

In [ ]:
train_1 = train.drop(['num_sold'] , axis = 1)
train_1_encoded = pd.get_dummies(train_1, columns = ['country' , 'store' , 'product'])
train_1['country'].value_counts()

In [ ]:
train_1_encoded = train_1_encoded.drop(['country_Sweden'] , axis =1)
train_1_encoded.head()

In [ ]:
train_1_all_encoded = train_1_encoded.drop(['store_KaggleMart' ,'product_Kaggle Hat'] , axis = 1)
train_1_all_encoded.head()

In [ ]:
x = train_1_all_encoded.copy()
x.head()

In [ ]:
train_1_all_encoded.reset_index(drop = True , inplace = True)
train_1_all_encoded.head(1)

In [ ]:
test_all_encoder = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')

test_all_encoder['Month'] = test_all_encoder['date'].apply(split_month)
test_all_encoder['Day'] = test_all_encoder['date'].apply(split_day)
test_all_encoder['Weekend'] = weekend(test_all_encoder['date'])
test_all_encoder['Weekday'] = weekday(test_all_encoder['date'])

test = pd.get_dummies(test_all_encoder , columns = ['country' , 'store' , 'product'])
test.drop(['date', 'country_Sweden', 'store_KaggleMart', 'product_Kaggle Hat', 'row_id'] , axis = 1 , inplace = True)
test.head()

In [ ]:
x['Month'] = x['Month'].astype('str').astype('int')
x['Day'] = x['Day'].astype('str').astype('int')

x.dtypes

In [ ]:
y = train['num_sold']

In [ ]:
from catboost import CatBoostRegressor
model = CatBoostRegressor()
model.fit(x,y)

In [ ]:
test['Month'] = test['Month'].astype('str').astype('int')
test['Day'] = test['Day'].astype('str').astype('int')

In [ ]:
prediction = model.predict(test)

submission = pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv' )
submission['num_sold'] = prediction
submission.to_csv('submission.csv', index=False) 
submission.head()